In [16]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env")
file_path = os.getenv("FILEPATH")

df = pd.read_csv(file_path)

In [17]:
# Simple Manupulation of Dataframe
df = df[df['Type'] == "Sale"].copy() 
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")
df["Amount"] = df["Amount"].abs()
df["Was Venmoed"] = " "
df["Split Count"] = df.get("Split Count", 1)

# Clean up Description field
df["Description"] = (
    df["Description"]
    .astype(str)
    .str.replace(r"#\d+.*", "", regex=True)
    .str.strip()
)
df.head(3)

,Transaction Date,Post Date,Description,Category,Type,Amount,Was Venmoed,Split Count
0,2025-10-28,10/29/2025,FSP*ROYAL OAK GOLF CENTER,Health & Wellness,Sale,14.50,,1
1,2025-10-27,10/28/2025,COSTCO WHSE,Shopping,Sale,139.79,,1
3,2025-10-25,10/27/2025,THE HOME DEPOT 2724,Home,Sale,58.79,,1
